In [ ]:
import numpy as np
from imgaug import augmenters as iaa
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

In [ ]:
images_train = np.load("images_dataset1.npy")
angles_train = np.load("angles_dataset1.npy") * 70
images_val = np.load("images_validation.npy")
angles_val = np.load("angles_validation.npy") * 70

In [ ]:
my_transform = transforms.Compose([transforms.ToPILImage(),
                                   transforms.Resize(size = (66,200)),
                                   transforms.ToTensor()
                                   
                                  ])

In [ ]:
from torch.utils.data.dataset import Dataset
from torchvision import transforms
class MyDataset(Dataset):
    def __init__(self, data, target, transform=my_transform):
        self.data = data
        self.target = torch.from_numpy(target)
        self.transform = transform
        st = lambda aug: iaa.Sometimes(0.4, aug)
        oc = lambda aug: iaa.Sometimes(0.3, aug)
        rl = lambda aug: iaa.Sometimes(0.09, aug)

        self.seq = iaa.Sequential([
        rl(iaa.GaussianBlur((0, 1.5))), # blur images with a sigma between 0 and 1.5
        rl(iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05), per_channel=0.5)), # add gaussian noise to images
        oc(iaa.Dropout((0.0, 0.10), per_channel=0.5)), # randomly remove up to X% of the pixels
        oc(iaa.CoarseDropout((0.0, 0.10), size_percent=(0.08, 0.2),per_channel=0.5)), # randomly remove up to X% of the pixels
        oc(iaa.Add((-40, 40), per_channel=0.5)), # change brightness of images (by -X to Y of original value)
        st(iaa.Multiply((0.10, 2.5), per_channel=0.2)), # change brightness of images (X-Y% of original value)
        rl(iaa.LinearContrast((0.5, 1.5), per_channel=0.5)), # improve or worsen the contrast
        rl(iaa.Grayscale((0.0, 1))), # put grayscale
        ],random_order=True)
     
    def __len__(self):
        return len(self.target)
    
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        x = self.transform(self.seq.augment_image(x))
        
        return x, y

In [ ]:
dataset_train = MyDataset(images_train,angles_train,transform=my_transform)
dataset_valid = MyDataset(images_val,angles_val,transform=my_transform)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
data_loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=64, shuffle=True, num_workers=0)
data_loader_valid = torch.utils.data.DataLoader(dataset=dataset_valid, batch_size=32, shuffle=True, num_workers=0)

In [ ]:
#######used to visualize images after transforming to tensor########
def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')

    return ax

In [ ]:
for batch_idx, (data, target) in enumerate(data_loader_train):
    target = target.view(-1,1)
    k = data
    print(data.shape)
    print(min(target))
    print(target[0])
    imshow(data[20],normalize=False)
    break

/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/color.py:548: UserWarning: Received an image with shape (H, W, C) and C=320 in ChangeColorspace._augment_image(). Expected C to usually be 3 -- any other value will likely result in errors. (Note that this function is e.g. called during grayscale conversion and hue/saturation changes.)
  "changes.)" % (image.shape[2],)


error: ignored

In [ ]:
def conv(in_channels, out_channels, kernel_size, stride=2, padding = 0,batch_norm=True):
    """Creates a convolutional layer, with optional batch normalization.
    """
    layers = []
    conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, 
                           kernel_size=kernel_size, stride=stride, padding = padding  ,bias=False)
    
    layers.append(conv_layer)

    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
    return nn.Sequential(*layers)

In [ ]:
class DriverNet(nn.Module):

  def __init__(self):
        super(DriverNet, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.BatchNorm2d(3),
            nn.Conv2d(3, 24, kernel_size=5, stride=2),
            nn.ELU(),
            conv(24, 36, kernel_size=5, stride=2),
            nn.ELU(),
            conv(36, 48, kernel_size=5, stride=2),
            nn.ELU(),
            conv(48, 64, kernel_size=3, stride=1),
            nn.ELU(),
            conv(64, 64, kernel_size=3, stride=1),
            nn.ELU(),
            nn.Dropout(p=0.5)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=64*1*18, out_features=100),
            nn.ELU(),
            nn.Dropout(p=0.4),
            nn.Linear(in_features=100, out_features=64),
            nn.ELU(),
            nn.Linear(in_features=64, out_features=10),
            nn.ELU(),
            nn.Linear(in_features=10, out_features=1)
        )
        

  def forward(self, x):
      #x = x.view(x.size(0), 3, 66, 200)
      output = self.conv_layers(x)
      output = output.view(output.size(0), -1)
      output = self.linear_layers(output)
      return output
model = DriverNet()

In [ ]:
import torch.optim as optim

### TODO: select loss function
criterion = nn.MSELoss()

### TODO: select optimizer
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    model = model.cuda()

In [ ]:

def train(n_epochs, data_loader_train ,data_loader_valid, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    k = np.Inf
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(data_loader_train):
            target = target.view(-1,1)
            # move to GPU
            if use_cuda:
                data, target = data.float().cuda(), target.cuda()
            optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
        # calculate the batch loss
            loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
        # perform a single optimization step (parameter update)
            optimizer.step()
        # update training loss
            train_loss += loss.item()*data.size(0)
        
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(data_loader_valid):
            target = target.view(-1,1)
            # move to GPU
            if use_cuda:
                data, target = data.float().cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
        # calculate the batch loss
            loss = criterion(output, target)
        # update average validation loss 
            valid_loss += loss.item()*data.size(0)
            
        # print training/validation statistics
        train_loss = train_loss/len(data_loader_train.sampler)
        valid_loss = valid_loss/len(data_loader_valid.sampler)
        if k > valid_loss:
            torch.save(model.state_dict(), save_path)
            print("Saving model.........")
            k = valid_loss

        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))

        
        
    return model




In [ ]:
path = '/media/gp/GP/ML/carla/nividia model/try.pt'
model_scratch = train(50, data_loader_train,data_loader_valid, model, optimizer, 
                      criterion, use_cuda, path)

/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/color.py:548: UserWarning: Received an image with shape (H, W, C) and C=320 in ChangeColorspace._augment_image(). Expected C to usually be 3 -- any other value will likely result in errors. (Note that this function is e.g. called during grayscale conversion and hue/saturation changes.)
  "changes.)" % (image.shape[2],)


error: ignored